In [62]:
import sys

import plotly.offline as py
import plotly.graph_objs as go
import plotly.figure_factory as FF

import numpy as np
import pandas as pd
from pandas.tseries.offsets import *

REPORT_FILENAME = 'sb-load.html'
N_REPLICAS = 8

if len(sys.argv) >= 2:
    N_REPLICAS = int(sys.argv[1])

In [20]:
perfmon_df = pd.read_csv('perfmon.jtl').loc[lambda df: df.label.str.endswith('CPU'), :]
kpi_df = pd.read_csv('kpi.jtl')

In [21]:
def parse_timestamp(df):
    df.index = df['timeStamp'].apply(pd.to_datetime, unit='ms')

perfmon_df = perfmon_df.rolling(100).mean()
parse_timestamp(perfmon_df)
perfmon_df['CPU'] = perfmon_df['elapsed'] / 1000

kpi_df = kpi_df.sort_values('timeStamp')
parse_timestamp(kpi_df)
kpi_df['Latency'] = kpi_df['Latency'] / 1000
searches_df = kpi_df.loc[lambda df: df.label.str.startswith('search'), :].loc[lambda df: df.responseCode == 200, :]
searches_per_min_df = searches_df.resample('60s').count() / N_REPLICAS
errors_per_min_df = kpi_df.loc[lambda df: df.responseCode != 200, :].resample('60s').count()

In [31]:
n_threads = kpi_df['allThreads'].resample('60s').mean()
searches_latency_df = (searches_df['elapsed'] / 1000).resample('60s')

In [66]:
duration = (kpi_df.index[-1:] - kpi_df.index[0])[0] / np.timedelta64(1, 's')
n_samples = len(kpi_df)
n_searches = len(searches_df)
throughput = n_samples / duration / N_REPLICAS
print("duration: %.2f sec" % duration)
print("thoughput: %.2f tps" % throughput)

duration: 2547.35 sec
thoughput: 3.89 tps


In [67]:
cpu_load = go.Scatter(x=perfmon_df.index, y=perfmon_df['CPU'],
                      mode='lines',
                      name='Average CPU%',
                      line=go.Line(color='#B94CE1'))

jmeter_threads = \
    go.Bar(x=n_threads.index,
           y=n_threads,
           name='JMeter Threads',
           marker=dict(color='#FF952B'))

errors = \
    go.Scatter(x=errors_per_min_df.index,
               y=errors_per_min_df['Latency'],
               mode='lines',
               name='Errors',
               line=go.Line(color='#FF2F2F'))

search_rpm = \
    go.Scatter(x=searches_per_min_df.index,
               y=searches_per_min_df['elapsed'],
               mode='lines',
               name='Searches/Min')

latency_upper = \
    go.Scatter(x=searches_latency_df.mean().index,
               y=searches_latency_df.max(),
               line=go.Line(color='transparent'),
               showlegend=False,
               name='Max')

latency_lower = \
    go.Scatter(x=searches_latency_df.mean().index,
               y=searches_latency_df.min(),
               line=go.Line(color='transparent'),
               fill='tonexty', 
               fillcolor='rgba(0,176,246,0.2)',
               showlegend=False,
               name='Min')

latency_mean = \
    go.Scatter(x=searches_latency_df.mean().index,
               y=searches_latency_df.mean(),
               mode='lines',
               line=go.Line(color='rgb(0,176,246)'),
               name='Search Latency, mean')
   
fig = go.Figure(data=[cpu_load, jmeter_threads, search_rpm, errors,
                      latency_upper, latency_lower, latency_mean],
                layout={
                    'xaxis': {'title': 'Throughput per instance: %.2f tps' % throughput},
                })
py.plot(fig, filename=REPORT_FILENAME)

'file:///Users/drdivano/jupyter/sb-load.html'